In [ ]:
from aicsimageio import AICSImage
import napari
from aicsimageio.readers import CziReader
import cv2 as cv
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
#VISUALIZATION
# visualize the image with napari using its numpy array
def visualize_napari(numpy_img: np.ndarray,name):
    """
    :param numpy_img: image to be visualized
    """
    with napari.gui_qt():
        viewer = napari.Viewer()
        viewer.add_image(numpy_img,name=name)

# visualize different images in the same moment
def visualize_all_list_napari(numpy_img_list: np.ndarray,names):
    """
    :param numpy_img_list: list containing different images to be visualized
    """
    with napari.gui_qt():
        viewer = napari.Viewer()
        for i, img in enumerate(numpy_img_list):
            viewer.add_image(img, name=names[i] )



In [ ]:
# Load the image
reader = CziReader("TB_sample\extern_Synlab_2156_17_3_MTB.czi")
# Get whole image
smear = reader.get_image_data("MYX", C=0)

In [ ]:
def is_blurry(image):
    #compute laplacian of image
    laplacian = cv.Laplacian(image, cv.CV_64F)
    #compute variance of laplacian
    measure=laplacian.mean()
    #print(measure)
    return measure

In [ ]:

def get_gradient_magnitude(image):
    #get the gradient magnitude
    sobelx = cv.Sobel(image, cv.CV_64F, 1, 0, ksize=3)
    sobely = cv.Sobel(image, cv.CV_64F, 0, 1, ksize=3)
    magnitude = np.sqrt(sobelx ** 2 + sobely ** 2)
    return magnitude

In [ ]:
def sobel(img):
    scale=1
    delta=0
    depth= cv.CV_16S

    grad_x = cv.Sobel(img, depth, 1, 0, ksize=3, scale=scale, delta=delta, borderType=cv.BORDER_DEFAULT)
    grad_y = cv.Sobel(img, depth, 0, 1, ksize=3, scale=scale, delta=delta, borderType=cv.BORDER_DEFAULT)
    abs_grad_x = cv.convertScaleAbs(grad_x)
    abs_grad_y = cv.convertScaleAbs(grad_y)
    grad = cv.addWeighted(abs_grad_x, 0.5, abs_grad_y, 0.5, 0)
    return grad

In [ ]:
def is_definitely_blurry(image):
    mod=get_gradient_magnitude(image)
    mes = mod.mean()
    #take into account that images vary in how much white they have
    return mes

In [ ]:
def is_definitely_blurry2(image):
    mod=get_gradient_magnitude(image)
    mes = mod.mean()
    #take into account that images vary in how much white they have
    return mes / image.sum()

In [ ]:
def is_blurry_fft(image):
    #convert image to grayscale

    #find fourier transform
    f = np.fft.fft2(image)
    fshift = np.fft.fftshift(f)
    magnitude_spectrum = 20*np.log(np.abs(fshift))
    #find mean of magnitude spectrum
    mean = np.mean(magnitude_spectrum)
    #print(mean)
    #if mean is less than 100 then image is blurry
    if mean < 220:
        return True
    else:
        return False
#plot spectrum of fft
def plot_spectrum(image):
    #convert image to grayscale

    #find fourier transform
    f = np.fft.fft2(image)
    fshift = np.fft.fftshift(f)
    magnitude_spectrum = 20*np.log(np.abs(fshift))
    #find mean of magnitude spectrum
    mean = np.mean(magnitude_spectrum)
    #print(mean)
    #if mean is less than 100 then image is blurry
    if mean < 200:
        print('blurry')
    else:
        print('not blurry')
    #plot spectrum
    plt.subplot(121),plt.imshow(image, cmap = 'gray')
    plt.title('Input Image'), plt.xticks([]), plt.yticks([])
    plt.subplot(122),plt.imshow(magnitude_spectrum, cmap = 'gray')
    plt.title('Magnitude Spectrum'), plt.xticks([]), plt.yticks([])
    plt.show()
    #save image

In [ ]:
def rescale_image(image):
    image = image.astype(np.float32)
    image -= image.min()
    image *= (255.0/image.max())
    return image.astype(np.uint8)

In [ ]:
img=smear[800]
#a=is_blurry2(img)
img = rescale_image(img)
magn=get_gradient_magnitude(img)
mes= is_definitely_blurry(img)
print(mes)

visualize_all_list_napari([img,magn],["og","edges"])

In [ ]:
print(type(smear))
mes=[]
for tile in smear:
    tile = rescale_image(tile)
    re=is_definitely_blurry(tile)
    #orders other way around
    print("w/o sum", re)
    re2= is_definitely_blurry2(tile)
    print(" w/ sum" , re2)
    mes.append(re2)

#get mean of list
g=np.array(mes)
s=np.sort(g)
f=g.argsort()
a=smear[f[0]]
b=smear[f[1]]
new_smear =  np.stack((a,b), axis=0)




print(smear.shape)
print(new_smear.shape)

for i in range(2,g.shape[0]):
    print(i)
    if i % 10 ==0:
        new_smear = np.concatenate((new_smear, [smear[f[i]]]), axis= 0)

In [ ]:
 print(new_smear.shape)
for i in range(0,len(s),10):
  print(s[i]  )
visualize_napari(new_smear,'ff')
